<a href="https://colab.research.google.com/github/Kaproc/An-lisis-Opinion-Git/blob/main/src/analisis_sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>